In [1]:
import math
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import numpy as np

In [2]:
df = pd.read_csv('train_data.csv')
X = df.drop(columns=['diagnosis']).to_numpy().T
y = df['diagnosis'].to_numpy().reshape(-1, 1).T

print(y)
print(y.shape)
print(X)
print(X.shape)

[[1 0 0 1 1 1 0 1 1 1 1 1 0 1 0 0 0 0 0 0 0 0 1 0 1 0 1 0 1 1 0 0 1 1 0 0
  0 1 0 0 1 0 0 1 0 1 0 0 0 0 0 0 1 1 1 1 1 0 1 0 1 1 0 0 0 1 0 0 1 0 0 0
  1 0 0 1 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 1 0 0 1 1 0 0 0 0 0 1 0 0 0 1 0
  1 1 1 0 0 0 0 0 1 0 0 1 1 0 1 0 1 0 0 1 0 0 1 1 0 0 0 0 1 0 1 0 0 1 1 0
  1 1 1 0 1 0 0 1 1 0 0 0 1 0 0 0 0 0 0 0 0 1 0 0 0 0 1 0 0 0 0 0 1 0 1 1
  1 0 0 1 1 0 0 1 0 1 0 1 0 1 0 1 1 0 0 1 0 0 0 0 0 0 0 1 1 1 0 0 1 0 1 0
  0 1 0 1 0 1 1 1 0 0 1 0 0 1 1 0 0 1 1 1 0 1 0 0 0 0 0 0 0 1 1 0 1 0 0 1
  1 0 0 0 1 1 0 0 0 1 0 0 0 1 0 0 1 0 1 0 0 0 0 1 0 0 0 0 0 1 0 1 0 0 1 0
  1 1 1 0 1 1 1 0 0 0 0 1 1 1 0 0 0 0 1 1 1 0 1 0 1 0 1 1 1 0 1 0 0 0 1 0
  0 0 1 0 0 1 0 0 0 1 0 0 1 1 1 0 1 1 0 0 0 0 1 0 1 0 0 1 0 0 1 0 0 0 0 0
  0 0 1 0 0 0 0 1 1 1 0 1 0 1 0 0 1 1 0 0 1 0 1 1 1 0 0 1 0 0 1 0 0 1 0 1
  0 0 0 0 1 0 0 0 1 0 1 0 1 0 1 1 1 0 1 0 0 0 0 1 0 1 0 0 0 0 0 0 0 1 0 0
  0 0 0 1 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 1]]
(1, 455)
[[ 0.51855873 -0.51636409 -0.36811839 ... -0.32616206

In [3]:
def sigmoid(z, derivative=False):
    if derivative:
        return z * (1 - z)
    return 1 / (1 + np.exp(-z))

In [4]:
def softmax(z):
    exp_z = np.exp(z - np.max(z, axis=0, keepdims=True))
    return exp_z / np.sum(exp_z, axis=0, keepdims=True)

In [5]:
def network_archi(X, y):
    n_input = X.shape[0]
    n_output = y.shape[0]
    n_hidden1 = math.ceil((n_input + n_output) / 2)
    n_hidden2 = math.ceil((n_input + n_output) / 4)
    return n_input, n_hidden1, n_hidden2, n_output

print(network_archi(X, y))

(30, 16, 8, 1)


In [6]:
def define_model(X, y):
    n_input, n_hidden1, n_hidden2, n_output = network_archi(X, y)
    
    W1 = np.random.randn(n_hidden1, n_input) * 0.01
    b1 = np.zeros((n_hidden1, 1))
    W2 = np.random.randn(n_hidden2, n_hidden1) * 0.01
    b2 = np.zeros((n_hidden2, 1))
    W3 = np.random.randn(n_output, n_hidden2) * 0.01
    b3 = np.zeros((n_output, 1))
    
    parameters = {'W1': W1, 'b1': b1,
                  'W2': W2, 'b2': b2,
                  'W3': W3, 'b3': b3}
    return parameters

params = define_model(X, y)
print(params)


{'W1': array([[-1.17757618e-02, -6.74338973e-03,  5.16788244e-03,
        -1.51415415e-02,  4.93377840e-03, -3.06823076e-03,
         8.20289086e-03,  3.08427081e-03,  3.82263454e-04,
         1.64871585e-02,  2.28724825e-02, -1.06918253e-02,
        -1.15862808e-02, -7.65378540e-03, -1.01008808e-02,
        -9.94928782e-04,  1.03577446e-02, -3.09102252e-03,
         3.51892168e-03, -2.66059877e-03,  4.01740596e-03,
        -4.28981105e-03, -2.48056539e-03,  1.49651684e-03,
        -3.24786121e-03,  2.63469732e-04,  1.76181224e-02,
         1.08626744e-02, -1.83905700e-03,  1.78525603e-02],
       [ 9.84893807e-03,  7.12576854e-03, -5.51287225e-03,
        -3.97199046e-02,  1.23030862e-02, -9.50349954e-03,
         2.95636423e-03, -1.46470392e-02, -1.39200455e-02,
         1.86300880e-02,  7.73894198e-03,  9.43948744e-03,
        -1.12283003e-02,  9.94920452e-04, -5.17687389e-03,
         6.32334352e-03,  1.47293236e-02, -3.50330924e-03,
         5.55553594e-03, -4.37207485e-04, -2.760

In [9]:
def forward_propagation(X, parameters):
    W1, b1 = parameters['W1'], parameters['b1']
    W2, b2 = parameters['W2'], parameters['b2']
    W3, b3 = parameters['W3'], parameters['b3']
    
    Z1 = np.dot(W1, X) + b1
    A1 = sigmoid(Z1)
    Z2 = np.dot(W2, A1) + b2
    A2 = sigmoid(Z2)
    Z3 = np.dot(W3, A2) + b3
    A3 = sigmoid(Z3)
    
    cache = {'Z1': Z1, 'A1': A1,
             'Z2': Z2, 'A2': A2,
             'Z3': Z3, 'A3': A3}
    return cache

In [8]:
def backward_propagation(X, y, parameters, cache):
    m = X.shape[1]
    W1, b1 = parameters['W1'], parameters['b1']
    W2, b2 = parameters['W2'], parameters['b2']
    W3, b3 = parameters['W3'], parameters['b3']
    
    A1, A2, A3 = cache['A1'], cache['A2'], cache['A3']

    dZ3 = A3 - y
    dW3 = (1/m) * np.dot(dZ3, A2.T)

    grads = {'dW3': dW3, 'db3': db3,
             'dW2': dW2, 'db2': db2,
             'dW1': dW1, 'db1': db1}
    return grads